In [13]:
import torch

from transformers import (
    AutoConfig, 
    AutoModelForCausalLM,
    AutoTokenizer
)

from datasets import load_dataset

In [2]:
aime24 = load_dataset("Maxwell-Jia/AIME_2024")

In [3]:
aime24

DatasetDict({
    train: Dataset({
        features: ['ID', 'Problem', 'Solution', 'Answer'],
        num_rows: 30
    })
})

In [53]:
aime24['train'][3]

{'ID': '2024-I-3',
 'Problem': "Alice and Bob play the following game. A stack of $n$ tokens lies before them. The players take turns with Alice going first. On each turn, the player removes either $1$ token or $4$ tokens from the stack. Whoever removes the last token wins. Find the number of positive integers $n$ less than or equal to $2024$ for which there exists a strategy for Bob that guarantees that Bob will win the game regardless of Alice's play.",
 'Solution': "Let's first try some experimentation. Alice obviously wins if there is one coin. She will just take it and win. If there are 2 remaining, then Alice will take one and then Bob will take one, so Bob wins. If there are $3$, Alice will take $1$, Bob will take one, and Alice will take the final one. If there are $4$, Alice will just remove all $4$ at once. If there are $5$, no matter what Alice does, Bob can take the final coins in one try. Notice that Alice wins if there are $1$, $3$, or $4$ coins left. Bob wins if there ar

In [15]:
model_name = "Qwen/Qwen3-8B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [16]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [11]:
model.generation_config

GenerationConfig {
  "bos_token_id": 151643,
  "do_sample": true,
  "eos_token_id": [
    151645,
    151643
  ],
  "pad_token_id": 151643,
  "temperature": 0.6,
  "top_k": 20,
  "top_p": 0.95
}

In [54]:
prompt1 = "Please reason step by step, and put your final answer within \boxed{}. " 
prompt2 = "Answer the following question, and put your final answer within \boxed{}. "

prompt = prompt1 + aime24['train'][3]['Problem']

In [59]:
prompt

"Please reason step by step, and put your final answer within \x08oxed{}. Alice and Bob play the following game. A stack of $n$ tokens lies before them. The players take turns with Alice going first. On each turn, the player removes either $1$ token or $4$ tokens from the stack. Whoever removes the last token wins. Find the number of positive integers $n$ less than or equal to $2024$ for which there exists a strategy for Bob that guarantees that Bob will win the game regardless of Alice's play."

In [56]:
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False # Switches between thinking and non-thinking modes. Default is True.
)

In [57]:
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

In [ ]:
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

In [ ]:
content = tokenizer.decode(output_ids, skip_special_tokens=True).strip("\n")
print("answer:", content)

In [52]:
box = 'boxed{'
box_begin = content.find(box)
box_end = box_begin + content[box_begin:].find('}')
content[box_begin+len(box):box_end]

'116'

In [67]:
model_name = "Qwen/Qwen2.5-Math-7B-Instruct"
device = "cuda:6" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="cuda:6",
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# prompt = 

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [76]:
# CoT
messages = [
    {"role": "system", "content": "Please reason step by step, and put your final answer within \boxed{}."},
    {"role": "user", "content": prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

In [77]:
model.device

device(type='cuda', index=6)

In [78]:
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=1024
)

In [82]:
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

content = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [83]:
content

' position \\( W \\) because the player can remove the last token.\n- \\( n = 2 \\) is a winning position \\( W \\) because the player can remove 1 token and leave the opponent with 1 token.\n- \\( n = 3 \\) is a winning position \\( W \\) because the player can remove 1 token and leave the opponent with 2 tokens.\n- \\( n = 4 \\) is a losing position \\( L \\) because the player can only remove 1 or 4 tokens, leaving the opponent with 3 or 0 tokens, both of which are winning positions.\n\nFor \\( n \\geq 5 \\), we can determine the position by considering the possible moves:\n- If \\( n \\) is a winning position, then there exists a move to a losing position.\n- If \\( n \\) is a losing position, then all possible moves lead to a winning position.\n\nUsing this, we can extend the pattern:\n- \\( n = 5 \\) is a winning position \\( W \\) because the player can remove 1 token and leave the opponent with 4 tokens.\n- \\( n = 6 \\) is a winning position \\( W \\) because the player can re

In [84]:
box = 'boxed{'
box_begin = content.find(box)
box_end = box_begin + content[box_begin:].find('}')
content[box_begin+len(box):box_end]

'404'